# CB2030 on the Wellness Study

## Intended Learning Outcomes (ILOs)

After this lab, you will be able to describe the concept of single nucleotide polymorphisms (SNP) and how they affect protein expression levels in eukaryotic systems. You will be able to identify and differentiate between genes that have a strong genetic linkage based on the measured protein level. After completing the optional Bonus Exercise, you will be able to implement several tools on the database. In which, you can apply with your own research or study.

## Introduction

This computer exercise has been designed to let you explore a unique dataset containing paired genetics and proteomics measurements in a healthy cohort. The original set includes whole genome sequencing data covering ~7.3 million SNPs with paired longitudinal data over 2 years. We will focus on the proteins that are present in human blood plasma, which is important for many biological processes and harbours common targets for diagnostics and therapy. It is therefore of great interest to understand the interplay between genetic and environmental factors to determine the specific protein levels in individuals. This can aid us in our understanding of the importance of genetic architecture related to the individual variability of plasma levels of proteins during adult life.

## Method

You will analyze a dataset combined of whole-genome sequencing, multiplex plasma protein profiling and clinical parameters. The dataset is artificial in the sense that we have masked and randomized parameters from the original dataset to not reveal any personal data.

>Note. We cannot share or publish any individual data. The data have been modified, imputed, and anonymized. The individual identities that you see in this exercises have been generated randomly from the most common names in Sweden. We do not intend to offend any particular person. Most of the exercise will be based on population-level and/or aggregated data. Please consider this when evaluating and interpreting the data from the biological point of view, when comparing results to published work etc. This may be the root cause of why you see discrepancies between the original publication and this exercise.

## Before starting the lab

Please carefully read these papers before the computer lab:
- [Whole-genome sequence association analysis of blood proteins in a longitudinal wellness cohort](https://genomemedicine.biomedcentral.com/articles/10.1186/s13073-020-00755-0)
- [Quantitative variability of 342 plasma proteins in a human twin population](https://pubmed.ncbi.nlm.nih.gov/25652787/)

<div class="alert alert-block alert-danger">
DONT FORGET TO UNZIP THE <b>data/syn.Genotype_annotation.txt.gz</b> FILE BEFORE STARTING THE LAB!
</div>

## Instructions for answering the questions

You don't have to create a report. Just answer the questions directly in this notebook. Write your answers in the markdown cells provided after each question (*Answer: *). If code is needed write it exactly bellow the question-answer markdown cell. If it's not there it will not be evaluated!

In [1]:
# Setup and read data
import os
import pandas as pd
import numpy as np
from utils import plot_manhattan, plot_location, plot_protein_visit
paths = {
    'Olink_assays': os.path.join("data", 'Olink_ensg_id_location.txt'),
    'Genotype_anno': os.path.join("data", 'syn.Genotype_annotation.txt'),
    'Olink_exprs': os.path.join("data", 'syn.Olink_exprs.txt'),
    'S2_FDA': os.path.join("data", 'S2.FDA.txt'),
    'pQTL_anno': os.path.join("data", 'pQTL.anno.csv')
}

def safe_read_csv(path, **kwargs):
    try:
        return pd.read_csv(path, **kwargs)
    except Exception as e:
        print(f'Could not read {path}: {e}')
        return pd.DataFrame()
    
# Olink assays
Olink_assays_anno = safe_read_csv(paths['Olink_assays'], sep='\t')
if not Olink_assays_anno.empty:
    Olink_assays_anno = Olink_assays_anno.drop_duplicates().reset_index(drop=True)

# Olink expression
Olink_exprs = safe_read_csv(paths['Olink_exprs'], sep='\t')
if not Olink_exprs.empty:
    Olink_exprs = Olink_exprs.melt(id_vars=['random_ID', 'visit'], var_name='Assay', value_name='NPX')
    Olink_exprs = Olink_exprs.merge(Olink_assays_anno, on='Assay', how='left')
    Olink_exprs['visit'] = Olink_exprs['visit'].apply(lambda x: f'visit{x}')

# SNP annotations
SNP_anno = safe_read_csv(paths['Genotype_anno'], sep='\t')

# FDA data
S2_FDA = safe_read_csv(paths['S2_FDA'], sep='\t')

# pQTL annotations
pQTL_anno = safe_read_csv(paths['pQTL_anno'], sep=',')
pQTL_filtered = pQTL_anno.copy()
idx = pQTL_filtered.groupby(['OlinkID', 'protein_gene_name'])['P'].idxmin()
pQTL_filtered = pQTL_filtered.loc[idx].reset_index(drop=True)

## Background and concept of eQTLs and pQTLs

### Q1.1-Q1.10 Please fill in the blank space from the list of available options below.
*Transcription Start Site (TSS) | Promoter | Lac operon | Repressor | Pseudogene | Activator | Small interfering RNA (siRNA) | Transfer RNA (tRNA) | Non-coding RNA | Non-coding RNA | Enhancer | MicroRNA (miRNA) | Post-translational modicfication (PTM) | RNA inteference (RNAi) | Silencer | Ribosomal RNA (rRNA) | Transcription factor (TF) | Inducer |*

RNA polymerase II (Pol II) is one of the most important proteins in eukaryotic cells. It binds
the DNA to start transcription at the ____Q1.1. Indeed, there are several proteins that can help transcription occur safely. We always call them ____Q1.2. Some of them can boost transcription, which we call ____Q1.3, and they bind to DNA at the ____Q1.4. On the contrary, gene expression also needs to be regulated with care and control in several circumstances. The proteins that do that job are called ____Q1.5, which are usually bound to DNA at a ____Q1.6 to reduce transcription. Importantly, an ____Q1.7 is a special molecule. It can either elevate or reduce transcription by binding to other molecules mentioned above. From now on, transcription can happen in multiple steps with much better control.

mRNA is a product of transcription, and we all want it. However, it may still not be stable. It can undergo ____Q1.8, which is also a biological process that regulates transcription with some other molecules, including some types of RNAs themselves. Accordingly, several genes may have been transcribed to mRNA, but not all
of them would be translated, and we call them ____Q1.9. On top of that, Pol II cannot recognise all genes. ____Q1.10 are located in the genome and look like a decent gene. They came from duplication or retrotransposition of a functional gene but never function. Hmmm, life is literally complicated at the cellular level but it always just goes on.

### Q1.2 - How many unique protein targets were detected? 
>Check either in the wellness study OR in the Olink_exprs dataframe

Answer:

### Q1.3 - How many subjects are involved in the experiment? 
>Check either in the wellness study OR in the Olink_exprs dataframe

Answer:

### Q1.4 - What percentage (%) of the analysed SNPs belong to a specific gene? Please show your calculation. 
>Check the SNP_anno dataframe for the answer

Answer:


## cis- and trans- pQTLs
> For Q2.1-2.8 use the pQTL_filtered dataframe

### Q2.1 - How many were intronic for cis?

Answer:

### Q2.2 - How many were intergenic for cis?

Answer:

### Q2.3 - How many were untranslated for cis?

Answer:

### Q2.4 - How many were found in the coding region for cis?

Answer:

### Q2.5 - How many were intronic for trans?

Answer:

### Q2.6 - How many were intergenic for trans?

Answer:

### Q2.7 - How many were untranslated for trans?

Answer:

### Q2.8 - How many were found in the coding region for trans?

Answer:

### Q2.9. - What are the characteristics of the cis- and trans-pQTLs?

Answer:


## Genetic locations

Run the plots and answer the questions. If you need more details because some text is not visible, please check the `pQTL_filtered` dataframe directly.

<div class="alert alert-block alert-danger">
Don't change anything in the code other than the p-value cutoff and the protein to plot in the visits-timeseries plot!
</div>

In [2]:
pvalue_cutoff = 6e-11  # select your p-value cutoff

# DONT CHANGE -------------
manhattan_fig = plot_manhattan(pQTL_anno, pvalue_cutoff)
manhattan_fig.show()
location_fig = plot_location(pQTL_anno, pvalue_cutoff)
location_fig.show()
#--------------------------

In [3]:
protein_name = "BST1"  # Select your protein to be plotted

# DONT CHANGE -------------
fig = plot_protein_visit(Olink_exprs, SNP_anno, pQTL_anno, protein_name)
fig.show()
#--------------------------

### Q3.1 - How were the trans-pQTL variants defined in the original paper. [2 sentences]

Answer:

### Q3.2 - Rank the 3 strongest trans-acting elements in this study? (Answer with their protein names)

Answer:

### Q3.3 - What is the least significant p-value of these three genes?

Answer:

### Q3.4 - How many cis-acting are considered if we use this p-value as cutoff, which only identifies 3 trans variants?

Answer:

### Q3.5 - Let us explore the genetic data for the three most significant trans-genes. From what chromosomes are these three proteins expressed?

Answer:

### Q3.6 - On what chromosomes are their significant corresponding pQTLs present? [5 sentences]

Answer:

### Q3.7 - By investigating the longitudinal protein data, which genotype is the most abundant one in the most significant trans-pQTL?

Answer:

### Q3.8 - Compare this to the second significant trans-pQTL, which genotype is the least abundant one?

Answer:

### Q3.9 - Compare this to the third significant trans-pQTL, which genotype is the least abundant one?

Answer:

### Q3.10 - Can you give one example of one trans-element (SNP ID) and its corresponding Protein name that lies within the same chromosome aka. “Hotspot”? 
>Use pQTL_filtered dataframe

Answer:



## False Discovery Rate (FDR) and p-values

False discovery rate correction is a method of conceptualizing the rate of type I errors in null hypothesis testing when conducting multiple comparisons. If you repeat a test enough times, you will always get a number of false positives. One of the goals of multiple testing is to control the FDR: the proportion of these erroneous results.

>In this part, please refer to the data in pQTL_filtered dataframe

### Q4.1 - How many significant hits were reported in the analysis using a cutoff of (p-value = 6x10-11)?	

Answer:

### Q4.2 - What is the difference between FDR (Benjamini and Hochberg) and Bonferroni False Positive rate?

Answer:

### Q4.3 - How many True Positives are we expecting in the dataset when using a q-value cutoff of 0.01?	

Answer:

### Q.4.5 - In the study, a total of 2936 associations reached a given statistical significance level (P < 6x10-11), why was this used as a cutoff?

Answer:


## Biological implication
Up to 80 FDA approved proteins are found in the list of FDA approved drug targets. You can check it by exploring the S2_FDA. 

### Q5.1 - Identify the FDA approved biomarker with the highest significant genetic regulation. (Hint: please elaborate between pQTLs from the wellness study and FDA biomarkers.) 

Answer:

### Q5.2 - Describe why this protein may cause problems when used as a biomarker. Give a short explanation to why this may be a problem. [200 words]

Answer:

### Q5.3 - Given this protein is a biomarker and high expression of this protein refers to high tendency of having disease, what genotype is at risk receiving a false positive test? How many patients are in this genotype?  [300 words]

Answer:


## Independent genetic variants
### Q.6 - Variants can be highly correlated due to linkage disequilibrium, how can you determine the independent genetic variants?

Answer:


## eQTL vs pQTL

### Q7.1 – Explain how you can investigate the casual mechanisms between eQTLs and pQTLs and how this can be explained by plasma protein levels? [150 words]

Answer:

### Q7.2 - What is the main difference between the wellness study and other pQTL studies? [150 words]

Answer:


---

# BONUS EXERCISES (Optional)

## Gene and disease

### BQ1.1 - What is the disease that you select, please explain the cause or etiology of the disease and its global impact?
>Choose whichever disease and gene you like! In some questions you will have to attach figures in the notebook. Please make sure you do that with: **![Example Image](path/to/your/example_image.png)**

Answer:

### BQ1.2 - Is there a possible treatment against it? How?

Answer:

### BQ1.3 - Identify 20 most related genes and the disease from literature. Hint: You can basically curate the gene list by literatures. 
>Please fill the linst bellow in this format: Gene name, Entrez Gene Symbol, ENSEMBL ID, Protein UNIPROT ID
- **Example:** Insulin, INS, ENSG00000254647, P01308
- 

## Gene Set Enrichment Analysis (GSEA)

1. Now, please to to : https://maayanlab.cloud/Enrichr/ Please read through the Background Information (https://maayanlab.cloud/Enrichr/help#background)
2.	Enter Entrez Gene Symbols into the field and press “submit”.
3.	You are now taken to a page where gene set enrichment results are displayed from various databases. Press “Ontologies” at the top of the page to view Gene Ontology results, and choose only one domain that mostly related to your disease (for example biological process). Click “Export entries to table”.

### BQ1 - Is there any other of the ontologies or other databases in Enrichr that you think would be helpful for this purpose? Motivate your answer. [max 200 words]

Answer:

4.	Next, we will visualize the GSEA results using Revigo (http://revigo.irb.hr), which reduces and combines GO-terms to simplify the interpretation. To use Revigo, we will have to make a choice of how to choose and/or give weight to the different GO-terms in our result, with the aim to give emphasis to the biological annotations that are more likely to represent the “true” relationship between the genes in our gene set.

### BQ2 - The resulting table contains all biological processes related to the submitted genes with related p-values and scores. Make a motivated choice of whether to use a subset of the results, or the complete results. Motivate your choice. If you decide to continue with a subset of the results, describe and motivate the choice of parameter(s) and cutoff(s) you use to do so. Is there any other valid strategy that could be employed instead of the one you chose? Why do you choose one strategy over the other? [max 200 words]

Answer:

5.	When you have selected the GO-terms you will need to extract the GO-term ID in the format of “GO:0007268” from the full term names. You can do that with a script or manually.
6.	Paste the GO-terms in the field. *Note: you can check from the examples it has bellow*. Optionally, p-values or scores can be pasted as a column in order to give genes different weight in the analysis. If another parameter than p-values are used, this is indicated by clicking “some other quantity, where [higher is better]”. After adding GO-term IDs (and optionally p-values), press “Start Revigo”. NB! If no visualizations are displayed: You might have to activate Flash for this website, install it, or switch browser. If this still does not work, you can download an R script for generating the TreeMap plot using the “Make R script for plotting treemaps” link under the TreeMap tab.

### BQ3 - In Revigo, we can visualize GO-terms to aid us in biological interpretation. GO-terms can be submitted unweighted, or assigned weight by p-values or scores. Make a choice whether to submit weighted or unweighted GO-terms, and how you set other parameters. Attach the treeplot here and describe and motivate your choice of strategy, and how it relates to your choice in task 2. [max 200 words]

Answer:

## Protein Network

### Now, it’s time to work with proteins. We will do it similarly with GSEA. Is there a PSEA (Protein Set Enrichment Analysis)? Yes, there is but it is not as popular as GSEA. Instead, we will use a tool called ReactomeGSA which can help us generate pathway analysis.
1.	Go to https://reactome.org/ . Click on the ‘Analysis Tools’ tab. You can now see that there are several tools available. We want to get the protein annotation on the Reactome network. Paste 20 protein Uniproit IDs in the list. 
>Note: you can check from the example on the side tab.
2.	Select ‘Project to human’ option. This step can reduce a process to map on the human reactomes. Lastly, click ‘Analyze’.
3.	You will now acquire the reactome network from your protein lists. Try to look around. Do we see any pathway related to the disease?

### BQ4 - Attach your reactome network here. What can you see from it? Please elaborate. [max 200 words]

Answer:

4.	We will move on to the last database of today which is based here in Stockholm, the Human Protein Atlas (HPA). As we have the list of genes and proteins now, we will see how they are associated within the human body. Go to www.proteinatlas.org.
5.	You can basically search with the gene list in the tab by adding the suffix ‘gene_names:’. For example ‘gene_name:ACE2, INS’ and so on. You will see your list of gene with information at different levels. You can download the tabular data of your query.

### BQ5 - Can you draw any conclusion from your search on HPA and your disease? Maybe, you should consider tissue specificity, subcellular location and diseases.

Answer:

## Inter-omics

### BQ6 - Can you please elaborate on what you see from GSEA, the reactome and HPA? [max 200 words]

Answer:

## The solutions

### BQ7 - From what you learn over multiple tools within systems biology, what is the impact of climate change and the disease? Do you think we can handle this global problem with systems biology? [max 200 words]
